In [1]:
import ehtim as eh
import numpy as np
import matplotlib.pyplot as plt
import ehtim.scattering.stochastic_optics as so
import sys
import os


def generate_scans(obs, averaging_mode, timeres=60.):

    if averaging_mode == 'scan-average':
        obs.add_scans(dt=2*timeres/3600.)
        obs = obs.avg_coherent(timeres)
        obs = obs.avg_coherent(0., scan_avg=True)
    elif averaging_mode == 'scan-subaverage':
        obs.add_scans(dt=2*timeres/3600.)
        obs = obs.avg_coherent(timeres)
    elif averaging_mode == 'snapshot':
        obs = obs.avg_coherent(timeres)
        obs.add_scans(dt=0.)
    else:
        raise ValueError('Averaging mode not recognized.')

    # Only include data points with at least 80% of averaging time  

    #todel = []
    #nscans = len(obs.tlist(scan_gather=True))
    ##print(nscans)
    #for i in range(nscans):
    #    if len(obs.tlist(scan_gather=True)[i]) < 3:
    #        todel.append(i)
    #        
    #obs.data = np.delete(obs.data, todel)
    #print(todel)
    #print(len(obs.tlist()))

    
    #Plot to check scans
    #nscans = len(obs.tlist(scan_gather=True))
    #print(nscans)
    #for i in range(nscans):
        #obs_scan = obs.copy()
        #obs_scan.data = obs_scan.tlist(scan_gather=True)[i]
        #plt.scatter(obs_scan.data['time'],np.zeros(len(obs_scan.data)))
    #plt.show()
    #input('...')

    return obs

def add_noise_deblur(obs, scan_number, noise_frac=0.05, refr_floor_frac=0.005, zbl=2.):

    obs_scan = obs.copy()
    obs_scan.data = obs_scan.tlist(scan_gather=True)[int(scan_number)]

    # Add noise
    # Fractional systematic noise
    obs_scan = obs_scan.add_fractional_noise(noise_frac)

    # Refractive noise floor
    # Should we estimate zbl from the data?
    obs_scan.data['sigma'] = (obs_scan.data['sigma']**2 + (refr_floor_frac*zbl)**2)**0.5

    # Deblur
    sm = so.ScatteringModel()
    obs_scan = sm.Deblur_obs(obs_scan.switch_polrep("stokes"))

    return obs_scan

#averaging_modes = ['scan-average', 'scan-subaverage', 'snapshot']

def makefile(uvfits_path):
    averaging_modes = ['scan-average']
    #noise_fracs = [0.02, 0.05]
    noise_fracs = [0.02]

    for averaging_mode in averaging_modes:
        for noise_frac in noise_fracs:
            # Set up paths
            simn = os.path.splitext(os.path.basename(uvfits_path))[0]
            obs_filename = uvfits_path.split('/')[-1].split('.uvf')[0]
            obs_dir = '/'.join(uvfits_path.split('/')[:-1])+"/" + simn
            out_dir = obs_dir + '/preprocessed_test/%s/noisefrac%s/'%(averaging_mode, noise_frac)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
    
            # Load data
            obs = eh.obsdata.load_uvfits(uvfits_path)
            print(len(obs.tlist()))
    
            # Divide into scans
            print(averaging_mode)
            obs_scans = generate_scans(obs, averaging_mode)
            
            # Add noise, collect scan info, and export scans
            nscans = len(obs_scans.tlist(scan_gather=True))
            print("Nscans: ", nscans)
            tstart = []
            tstop = []
            sites = []
            gs = []
            isc = -1
            for scan_number in range(nscans):
                obs_scan_processed = add_noise_deblur(obs_scans, scan_number, noise_frac=noise_frac)

                if len(obs_scan_processed.data["time"]) < 3:
                    print("Skipping scan", scan_number)
                    continue
                isc += 1
                gs.append(scan_number)
    
                tstart.append(obs_scan_processed.data['time'][0])
                tstop.append(obs_scan_processed.data['time'][-1])

                scan_sites = []
                for site in obs_scan_processed.tarr['site']:
                    if site in obs_scan_processed.data['t1'] or site in obs_scan_processed.data['t2']:
                        scan_sites.append(site)
                sites.append(scan_sites)
    
                out_file = out_dir + obs_filename + '_preprocessed_%s_noisefrac%s_scan%s.uvfits'%(averaging_mode, noise_frac, isc)
                obs_scan_processed.save_uvfits(out_file)
    
            # Write out scan info
            if noise_frac == noise_fracs[0]:
                with open(obs_dir + '/preprocessed_test/%s/scaninfo_%s.txt'%(averaging_mode, averaging_mode), 'w') as f:
                    print('# Scan number, tstart, tstop, sites', end='\n', file=f)
                    for i in range(len(sites)):
                        print('%03d %.5f %.5f %s'%(i, tstart[i], tstop[i], ','.join(sites[i])), end='\n', file=f)
                f.close()
    
    
    
    

Welcome to eht-imaging! v 1.2.2 



In [50]:
makefile("MoD_calibration_suite/dataset027.uvfits")

Loading uvfits:  MoD_calibration_suite/dataset027.uvfits
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
2029
scan-average


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


Nscans:  42
Skipping scan 0
Skipping scan 1
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Skipping scan 4
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No

In [7]:
for i in range(0,35):
    makefile("dataCal/uvfits_MoD_precal/dataset%03d.uvfits"%i)

Loading uvfits:  dataCal/uvfits_MoD_precal/dataset000.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
2030
scan-average


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


Nscans:  42
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No N

/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


Nscans:  42
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No N

...

Nscans:  42
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No N

/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


Nscans:  42
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No N

/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


Nscans:  41
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No NX table in saved uvfits
Building NX table
No N

In [2]:
d = "dataCal/uvfits_MoD_precal/dataset000.uvfits"

In [4]:
obs = eh.obsdata.load_uvfits(d)
obs.add_scans(dt=2*60.0/3600.)
obs = obs.avg_coherent(60.0)
obs = obs.avg_coherent(0., scan_avg=True)
len(obs.tlist(scan_gather=True))

Loading uvfits:  dataCal/uvfits_MoD_precal/dataset000.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


42

In [6]:
obs.tlist(scan_gather=True)[37]

array([(14.34166667, 0., 'AA', 'AP', 0., 0., -7.09163204e+05, -1.84344560e+06,  1.00070728+7.75095916e-04j, -0.05512815+0.00042985j, -0.0020806 +0.00325517j,  0.07772459+7.76482074e-05j, 0.00033336, 0.00033336, 0.00033336, 0.00033336),
       (14.34166667, 0., 'AA', 'AZ', 0., 0.,  2.01726130e+09, -5.12166006e+09,  0.06423977+6.67124385e-05j,  0.01810689+0.00342788j,  0.01785545-0.00783385j,  0.00096574+4.66631788e-04j, 0.00076643, 0.00076643, 0.00076643, 0.00076643),
       (14.34166667, 0., 'AA', 'JC', 0., 0.,  5.22781344e+09, -4.83186499e+09,  0.00632434-6.15467558e-04j,         nan       +nanj,         nan       +nanj,         nan           +nanj, 0.00076838,        nan,        nan,        nan),
       (14.34166667, 0., 'AA', 'SM', 0., 0.,  5.22784633e+09, -4.83193668e+09,  0.00706188-8.16778822e-05j,  0.00776578+0.00907209j,  0.00259232+0.00580764j,  0.00077555-3.26163953e-04j, 0.00051659, 0.00051659, 0.00051659, 0.00051659),
       (14.34166667, 0., 'AA', 'SP', 0., 0.,  4.40339770

In [52]:
todel = []
for i in range(len(obs.data)):
    if len(obs.data['time']):
        todel.append(i)
obs.data = np.delete(obs.data, todel)
print(todel)
print(len(obs.tlist()))


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [33]:
obs.data

array([( 4.05833333, 0., 'AA', 'PV', 0., 0., -3.53273975e+09, -5.59312004e+09,  4.13508898e-02+2.80117832e-04j, -7.92413652e-03-1.16137518e-02j, -2.39223686e-03+9.00546828e-03j, -5.04611733e-04-1.36748322e-03j, 0.00078029, 0.00078029, 0.00078029, 0.00078029),
       ( 4.05833333, 0., 'AA', 'SP', 0., 0., -4.39323994e+09,  3.09016378e+09,  9.14970838e-02-1.93550054e-03j,  4.84528177e-02+6.22667867e-02j, -6.66623608e-02-9.34212003e-03j,  2.22906067e-03-2.61928813e-04j, 0.00117241, 0.00117241, 0.00117241, 0.00117241),
       ( 4.05833333, 0., 'PV', 'SP', 0., 0., -8.60500194e+08,  8.68328377e+09,  1.02108822e-02+3.57131923e-02j,  1.02551066e-02-9.61948277e-05j, -7.17443705e-04-8.90130851e-03j, -8.62007165e-03+4.09435466e-03j, 0.0116718 , 0.0116718 , 0.0116718 , 0.0116718 ),
       ( 4.40833333, 0., 'AA', 'PV', 0., 0., -3.77647338e+09, -5.43014842e+09,  4.58279847e-02-3.78026559e-04j, -9.19717956e-03-1.35440666e-02j, -2.50089608e-03+1.02085875e-02j,  2.71599238e-03+5.18455437e-04j, 0.0007777

# PreImcal Version

In [46]:
import preimcal

In [147]:
?preimcal.preim_pipeline

In [164]:
import ehtim as eh
import numpy as np
import matplotlib.pyplot as plt
import ehtim.scattering.stochastic_optics as so
import sys
import os

def precal(obs,scan_avg=True, tint = 60.0, is_normalized=False, syserr=0.02):
    if scan_avg:
        print("Here")
        obscal = obsproc = preimcal.preim_pipeline(obs, is_normalized=is_normalized, 
                                                   syserr=syserr, do_psd_noise=False, 
                                                   ref_optype='quarter1', 
                                                   do_deblurr=True, 
                                                   ref_scale=1.0)
        
        obscal.add_scans(dt=2*60.0/3600.)
        obscal = obscal.avg_coherent(60.0)
        obscal = obscal.avg_coherent(0., scan_avg=True)
        return obscal.copy().avg_coherent(0.0, scan_avg=True)
    else:
        obscal = obsproc = preimcal.preim_pipeline(obs, is_normalized=is_normalized, 
                                                   tint=tint, syserr=syserr, 
                                                   do_psd_noise=False, 
                                                   ref_optype='quarter1',
                                                   do_deblurr=True, 
                                                   ref_scale=1.0)
        
        return obscal




def makefile_preimcal(uvfits_path, scan_avg=True, tint = 60.0, is_normalized=False, syserr=0.02):
    if scan_avg:
        averaging_modes = ['scan-average']
    else:
        averaging_modes = ["snapshot%02d"%tint]
    #noise_fracs = [0.02, 0.05]
    noise_fracs = [syserr]

    for averaging_mode in averaging_modes:
        for noise_frac in noise_fracs:
            # Set up paths
            simn = os.path.splitext(os.path.basename(uvfits_path))[0]
            obs_filename = uvfits_path.split('/')[-1].split('.uvf')[0]
            obs_dir = '/'.join(uvfits_path.split('/')[:-1])+"/" + simn
            out_dir = obs_dir + '/preimcal_pipeline/%s/noisefrac%s/'%(averaging_mode, noise_frac)
            if not os.path.exists(out_dir):
                os.makedirs(out_dir)
    
            # Load data
            obs = eh.obsdata.load_uvfits(uvfits_path)
            print(len(obs.tlist()))
    
            # Divide into scans
            print(averaging_mode)
            obs_proc = precal(obs, scan_avg=scan_avg, is_normalized=is_normalized, syserr=noise_frac).copy()
            # Add noise, collect scan info, and export scans
            if scan_avg:
                obs_scans = obs_proc.split_obs(scan_gather=True)
            else:
                obs_scans = obs_proc.split_obs()
            nscans = len(obs_scans)
            print("Nscans: ", nscans)
            tstart = []
            tstop = []
            sites = []
            gs = []
            isc = -1
            for scan_number in range(nscans):
                obs_scan = obs_scans[scan_number]
                print(len(obs_scan.data))


                if len(obs_scan.data["time"]) < 3:
                    print("Skipping scan", scan_number)
                    continue
                isc += 1
                out_file = out_dir + obs_filename + '_preprocessed_%s_noisefrac%s_scan%s.uvfits'%(averaging_mode, noise_frac, isc)
                obs_scan.save_uvfits(out_file)

                gs.append(scan_number)
    
                tstart.append(obs_scan.data['time'][0])
                tstop.append(obs_scan.data['time'][-1])

                scan_sites = []
                for site in obs_scan.tarr['site']:
                    if site in obs_scan.data['t1'] or site in obs_scan.data['t2']:
                        scan_sites.append(site)
                sites.append(scan_sites)
    
    
            # Write out scan info
            if noise_frac == noise_fracs[0]:
                with open(obs_dir + '/preimcal_pipeline/%s/scaninfo_%s.txt'%(averaging_mode, averaging_mode), 'w') as f:
                    print('# Scan number, tstart, tstop, sites', end='\n', file=f)
                    for i in range(len(sites)):
                        print('%03d %.5f %.5f %s'%(i, tstart[i], tstop[i], ','.join(sites[i])), end='\n', file=f)
                f.close()
    
    
    
    


# ScanAvg

In [153]:
for i in range(1,35):
    makefile_preimcal("dataCal/uvfits_MoD_precal_stage1/dataset%03d.uvfits"%i, is_normalized=True)

Loading uvfits:  dataCal/uvfits_MoD_precal_stage1/dataset001.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
1672
scan-average
Here
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging processing: deblurring data
Building NX table
Splitting Observation File into 36 times
Nscans:  36
3
Building NX table
3
Building NX table
6
Building NX table
6
Building NX table


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


3
Building NX table
3
Building NX table
6
Building NX table
6
Building NX table
8
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
21
Building NX table
15
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
15
Building NX table
15
Building NX table
Loading uvfits:  dataCal/uvfits_MoD_precal_stage1/dataset002.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
1664
scan-average
Here
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized 

/home/ptiede/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Pre-imaging processing: deblurring data
Building NX table
Splitting Observation File into 36 times
Nscans:  36
3
Building NX table
3
Building NX table
6
Building NX table


...

3
Building NX table
3
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
21
Building NX table
15
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
15
Building NX table
15
Building NX table
Loading uvfits:  dataCal/uvfits_MoD_precal_stage1/dataset034.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
1674
scan-average
Here
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized

/home/ptiede/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Pre-imaging processing: deblurring data
Building NX table
Splitting Observation File into 36 times
Nscans:  36
3
Building NX table
3
Building NX table
6
Building NX table
6


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


Building NX table
3
Building NX table
3
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
21
Building NX table
15
Building NX table
21
Building NX table
21
Building NX table
20
Building NX table
21
Building NX table
21
Building NX table
15
Building NX table
15
Building NX table


In [154]:
for i in range(36,119):
    makefile_preimcal("dataCal/uvfits_MoD_precal_stage2/dataset%03d.uvfits"%i, is_normalized=True)

Loading uvfits:  dataCal/uvfits_MoD_precal_stage2/dataset036.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
1662
scan-average
Here
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.


/home/ptiede/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Pre-imaging processing: deblurring data
Building NX table
Splitting Observation File into 36 times
Nscans:  36
1
Skipping scan 0
1
Skipping scan 1
3
Building NX table
3
Building NX table
1
Skipping scan 4
3
Building NX table
3
Building NX table


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


3
Building NX table
4
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
21
Building NX table
15
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
15
Building NX table
15
Building NX table
Loading uvfits:  dataCal/uvfits_MoD_precal_stage2/dataset037.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
1696
scan-average
Here
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.


...

3
Building NX table
3
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
10
Building NX table
9
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
21
Building NX table
15
Building NX table
21
Building NX table
15
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
15
Building NX table
14
Building NX table
1
Skipping scan 36
5
Building NX table
Loading uvfits:  dataCal/uvfits_MoD_precal_stage2/dataset118.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
1713
scan-average
Here
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the r

/home/ptiede/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Pre-imaging processing: deblurring data
Building NX table
Splitting Observation File into 37 times
Nscans:  37
3
Building NX table
3
Building NX table
6
Building NX table
6
Building NX table


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


3
Building NX table
3
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
21
Building NX table
1
Skipping scan 28
6
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
15
Building NX table
15
Building NX table


In [143]:
makefile_preimcal("SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_10s.uvfits")

Loading uvfits:  SgrAData/SgrA_ER6_ver2021-04-15/3599/hops/hops_3599_SGRA_LO_netcal_LMTcal_10s.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
2061
scan-average
Here
Pre-imaging processing: Obtaining the light curve from the intra-site baselines
Pre-imaging processing: Normalizing the total flux density
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.


/home/ptiede/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Pre-imaging processing: deblurring data
Pre-imaging processing: rescaling the total flux density.


/home/ptiede/.local/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


Nscans:  43
Splitting Observation File into 43 times
3
Building NX table
3
Building NX table
6
Building NX table
6
Building NX table
3
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
4
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
6
Building NX table
10
Building NX table
10
Building NX table
10
Building NX table
15
Building NX table
15
Building NX table
10
Building NX table
21
Building NX table
10
Building NX table
15
Building NX table
21
Building NX table
21
Building NX table
21
Building NX table
10
Building NX table
21
Building NX table
21
Building NX table
15
Building NX table
15
Building NX table
10
Building NX table
6
Building NX table
6
Building NX table
6
Building NX table
3
Building NX table


# Snapshot

In [165]:
for i in range(1,2):
    makefile_preimcal("dataCal/uvfits_MoD_precal_stage1/dataset%03d.uvfits"%i, is_normalized=True, scan_avg=False)

Loading uvfits:  dataCal/uvfits_MoD_precal_stage1/dataset001.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
1672
snapshot60
Pre-imaging processing: time-averaging.
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging processing: deblurring data
Splitting Observation File into 291 times
Nscans:  291
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits

KeyboardInterrupt: 

In [166]:
for i in range(1,2):
    makefile_preimcal("dataCal/uvfits_MoD_precal_stage1/dataset%03d.uvfits"%i, is_normalized=True, scan_avg=False, tint=120)

Loading uvfits:  dataCal/uvfits_MoD_precal_stage1/dataset001.uvfits
no IF in uvfits header!
POLREP_UVFITS: circ
Number of uvfits Correlation Products: 4
No NX table in uvfits!
1672
snapshot120
Pre-imaging processing: time-averaging.
Pre-imaging processing: adding the fractional systematic errors.
Pre-imaging processing: adding the refractive noise to the flux-normalized data.
Pre-imaging processing: deblurring data
Splitting Observation File into 291 times
Nscans:  291
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfits
3
Building NX table
No NX table in saved uvfit

KeyboardInterrupt: 